In [5]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator.types import SyntheticGraphType

# 1. Graph generation

In [6]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

In [7]:
# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

simple_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL,
                          cluster_counts=10,
                          bottom_border=100,
                          top_border=200)

In [8]:
# complex graph
# should generate general (average) type of graph with 300 unique works, 100 resources and 2000 vertices

advanced_wg = ss.advanced_work_graph(works_count_top_border=2000,
                                     uniq_works=300,
                                     uniq_resources=100)

# 2. Contractor generation

In [9]:
from uuid import uuid4
from sampo.schemas.resources import Worker
from sampo.schemas.contractor import Contractor

### Manual generation
To create contractor, you should provide minimal info: unique id, contractor name, and supplied workers (simple renewable resources).

In [10]:
contractors = [
    Contractor(id=str(uuid4()),
               name="OOO Berezka",
               workers={'worker' : Worker(id='0', name='worker', count=100)})
]

### Generation from graph

In [11]:
from sampo.generator.environment.contractor_by_wg import get_contractor_by_wg

contractors = [get_contractor_by_wg(simple_wg)]

# 3. Scheduling

### Scheduler construction
Before scheduling you should specify scheduling algorithm used for transforming input data to the final schedule.
At this time SAMPO contains heuristic algorithms, such as HEFTAddEnd, HEFTAddBetween and Topological scheduler, and the Genetic algorithm.
While creation, you can specify the hyperparameters to fit the algorithm.

In [12]:
from sampo.scheduler.heft.base import HEFTScheduler

# here we can just create simple heuristic scheduler
scheduler = HEFTScheduler()

In [13]:
from sampo.scheduler.genetic.base import GeneticScheduler

# or more complex genetic scheduler
scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3)

Can not find native module; switching to default


### Scheduling process
SAMPO provides a simple interface to all its features.
It called SchedulingPipeline.
Using it you only should pass all the scheduling arguments, it remembers them, and you can produce schedules in many ways.

In [14]:
from sampo.pipeline import SchedulingPipeline

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Genetic optimizing took 33.90812873840332 ms
Toolbox initialization & first population took 308.14647674560547 ms
First population evaluation took 522.6023197174072 ms
-- Generation 0, population=20, best time=995.0 --
-- Generation 1, population=22, best time=995.0 --
-- Generation 2, population=34, best time=989.0 --
-- Generation 3, population=37, best time=989.0 --
-- Generation 4, population=29, best time=989.0 --
-- Generation 5, population=39, best time=989.0 --
-- Generation 6, population=63, best time=989.0 --
-- Generation 7, population=34, best time=989.0 --
-- Generation 8, population=49, best time=989.0 --
-- Generation 9, population=45, best time=989.0 --
-- Generation 10, population=101, best time=983.0 --
-- Generation 11, population=46, best time=983.0 --
-- Generation 12, population=34, best time=983.0 --
-- Generation 13, population=49, best time=983.0 --
-- Generation 14, population=49, best time=983.0 --
-- Generation 15, population=95, best time=982.0 --
-- Genera

968

### Other metrics
Genetic scheduler can do more than just optimize MakeSpan. It supports more metrics.
Metrics are represented by `FitnessFunction` class and can be passed to `GeneticScheduler` on constructing.

Here we're constructing Genetic that should optimize resources to deadline.

In [15]:
from sampo.schemas.time import Time
from sampo.scheduler.genetic.operators import DeadlineResourcesFitness

deadline = Time(2000)
# calling `prepare` method to pass explicit parameters to fitness function
fitness_constructor = DeadlineResourcesFitness.prepare(deadline)

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Can keep deadline at minimum resources
Genetic optimizing took 32.93895721435547 ms
Toolbox initialization & first population took 159.57283973693848 ms
First population evaluation took 573.4663009643555 ms
-- Generation 0, population=20, best time=313.0 --
-- Generation 1, population=57, best time=298.0 --
-- Generation 2, population=56, best time=298.0 --
-- Generation 3, population=82, best time=273.0 --
-- Generation 4, population=100, best time=264.0 --
-- Generation 5, population=48, best time=243.0 --
-- Generation 6, population=54, best time=243.0 --
-- Generation 7, population=22, best time=243.0 --
-- Generation 8, population=20, best time=243.0 --
-- Generation 9, population=20, best time=243.0 --
-- Generation 10, population=20, best time=243.0 --
-- Generation 11, population=20, best time=243.0 --
-- Generation 12, population=20, best time=243.0 --
-- Generation 13, population=20, best time=243.0 --
-- Generation 14, population=20, best time=243.0 --
-- Generation 15, popu

1891

Additionally, you can construct other metrics: deadline cost, time with resources (without deadline) fitness:

In [16]:
from sampo.scheduler.genetic.operators import DeadlineCostFitness

fitness_constructor = DeadlineCostFitness.prepare(deadline)

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Can keep deadline at minimum resources
Genetic optimizing took 37.89687156677246 ms
Toolbox initialization & first population took 248.33416938781738 ms
First population evaluation took 672.2033023834229 ms
-- Generation 0, population=20, best time=1312150.0 --
-- Generation 1, population=20, best time=1312150.0 --
-- Generation 2, population=20, best time=1312150.0 --
-- Generation 3, population=20, best time=1312150.0 --
-- Generation 4, population=20, best time=1312150.0 --
-- Generation 5, population=20, best time=1312150.0 --
-- Generation 6, population=20, best time=1312150.0 --
-- Generation 7, population=20, best time=1312150.0 --
-- Generation 8, population=20, best time=1312150.0 --
-- Generation 9, population=20, best time=1312150.0 --
-- Generation 10, population=20, best time=1312150.0 --
-- Generation 11, population=20, best time=1312150.0 --
-- Generation 12, population=20, best time=1312150.0 --
-- Generation 13, population=20, best time=1312150.0 --
-- Generation 14, p

1891

In [17]:
from sampo.scheduler.genetic.operators import TimeAndResourcesFitness

fitness_constructor = TimeAndResourcesFitness

scheduler = GeneticScheduler(mutate_order=0.1,
                             mutate_resources=0.3,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

schedule = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()

schedule.execution_time

Can keep deadline at minimum resources
Genetic optimizing took 44.44313049316406 ms
Toolbox initialization & first population took 316.01643562316895 ms
First population evaluation took 551.9542694091797 ms
-- Generation 0, population=20, best time=1337.0 --
-- Generation 1, population=33, best time=1333.0 --
-- Generation 2, population=27, best time=1333.0 --
-- Generation 3, population=20, best time=1333.0 --
-- Generation 4, population=20, best time=1333.0 --
-- Generation 5, population=20, best time=1333.0 --
-- Generation 6, population=20, best time=1333.0 --
-- Generation 7, population=20, best time=1333.0 --
-- Generation 8, population=20, best time=1333.0 --
-- Generation 9, population=20, best time=1333.0 --
-- Generation 10, population=20, best time=1333.0 --
-- Generation 11, population=20, best time=1333.0 --
-- Generation 12, population=20, best time=1333.0 --
-- Generation 13, population=20, best time=1333.0 --
-- Generation 14, population=20, best time=1333.0 --
-- Gener

989